In [ ]:
import pandas as pd

data = pd.read_parquet('src/data/data')

In [ ]:
data['conversion'].unique()

In [ ]:
data.head()

In [ ]:
data['boosted'] = data['boosted'].apply(lambda x: int(x))
y = pd.get_dummies(data.platform, prefix='platform')
x = pd.get_dummies(data.device_type, prefix='device_type')

In [ ]:
data = pd.concat([data,y,x], axis=1)
df = data.query("taxonomy_level1=='Sports and Fitness'")
df

In [ ]:
from sklearn.preprocessing import OneHotEncoder
import math
from sklearn.linear_model import LogisticRegression

In [ ]:
d = {}
for i in data['taxonomy_level1'].unique().tolist():
    try:
        df = data.query(f"taxonomy_level1=='{i}'")
        df['price'] = df['price'].apply(lambda x: math.log(x))
        X = df[['boosted','free_shipping','is_pdp','price','print_position','platform_mobile','device_type_desktop','device_type_ios','device_type_mobile']]
        y = df['conversion']
        clf = LogisticRegression(random_state=0).fit(X, y)
        clf.score(X, y)
        coef = clf.coef_[0].tolist()[4]
        d[i] = coef
    except:
        pass


In [ ]:
df = data.copy()
df = df.query("taxonomy_level1 != 'Other categories'")
df['price'] = df['price'].apply(lambda x: math.log(x))
X = df[['boosted','free_shipping','is_pdp','original_price','price','print_position','platform_mobile','device_type_desktop','device_type_ios','device_type_mobile']]
y = df['conversion']
clf = LogisticRegression(random_state=0).fit(X, y)
clf.score(X, y)
coef = clf.coef_[0].tolist()[4]
d[i] = coef

print([round(x,2) for x in clf.coef_[0].tolist()])

In [ ]:
coef_df = pd.DataFrame({k: v*100 for k, v in sorted(d.items(), key=lambda item: item[1])}, index=['1']).T
coef_df.reset_index(inplace=True)
coef_df.columns=['taxonomy_level1','coef']


In [ ]:
import matplotlib.pyplot as plt
coef_df.iloc[:].plot.bar(x="taxonomy_level1", y="coef")
plt.show()

In [ ]:
clf = LogisticRegression(random_state=0).fit(X, y)
clf.score(X, y)
clf.coef_[0].tolist()[4]

In [ ]:
import statsmodels.api as sm
model = sm.OLS(y, X).fit()
print(model.summary())

In [ ]:
data['taxonomy_level1'].value_counts().reset_index()

In [ ]:
import plotly.express as px

violin_df = data.query("taxonomy_level1 in ('Home, Furniture and Garden','Clothes and accessories','Tools and Construction','Sports and Fitness','Beauty and Personal Care','Electronics, Audio and Video')")
print(violin_df)
fig = px.violin(violin_df, y="price", x="taxonomy_level1", color="conversion", box=True)
fig

In [ ]:
data.columns

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.pairplot(data)
plt.show()


In [ ]:
sns.heatmap(data.drop(columns=['boosted']).corr().round(2), annot=True, annot_kws={"size": 7}, linewidths=0.1, linecolor='black')